# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s):
###### Python version:
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [26]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filepath):
    '''
    Read file by filepath, and make it into a 2-D list
    _____________________
    Parameter:
    filepath -> The filepath of dataset in csv format
    ____________________
    Return:
    A 2-D list which contains the content of the dataset passed in
    '''
    # open the csv file in a reading mode
    csv_file = open(filepath, 'r')
    # The dataset for storing data
    dataset = []
    
    # read the csv file line by line
    for line in csv_file.readlines():
        # Each line of the csv file represents an instance
        instance = line.split(',')
        # The split method above would capture the newline character '\n' at end of the line
        # Let's remove "\n" by list slicing
        instance[-1] = instance[-1][:-1]
        # Append current instance to the final dataset
        dataset.append(instance)
    # Finishing reading, close the file
    csv_file.close()
    
    # return the final dataset 
    return dataset

In [47]:
# This function should build a supervised NB model
def train(dataset):
    '''
    Generate prior and posterir distribution from the give dataset.
    The class atrribute is assumed to be at the last column of each instance. 
    _______________________________
    Parameter:
    dataset -> the trainning dataset in the form of 2-D list
    _______________________________
    Return:
    prior -> A dictionary contains prior distribution of classes, that is P(cj)
    posterior -> A dictionary contains posterior, that is P(xi|cj)
    '''
    return prior_calculator(dataset), posterior_calculator(dataset)

In [ ]:
def prior_calculator(dataset):
    """
    Helper function for train(), responsible for calculating the prior of the given dataset
    _____________________
    Parameter:
    dataset -> given dataset for calculating prior
    _____________________
    Return:
    prior -> A dictionary represent the prior of the given dataset
    """
    
    prior = {}
    for instance in dataset:
        the_class = instance[-1]
         # if key is already seen before, just increase the count by 1
        if the_class in prior:
            prior[the_class] += 1
        # else create a new (key, value) pair and assign value to 1
        else:
            prior[the_class] = 1
    
    # divide the prior counts by total number to get probability
    n = len(dataset)
    for the_class in prior.keys():
        prior[the_class] /= n
    
    # All done, return the prior
    return prior

In [ ]:
def posterior_calculator(dataset):
    """
    This posterior follow the data structure disscussed in lecture
    {attribute x: 
                {class c: 
                            {attribute_value: value}
                }
    }
   
    The posterior gives the probability after "Laplace smoothing", that is all count has a initial value of 1,
    
    P(xi|cj) = (1 + value_count)/(class_count + distinct_value_count_of_attribute)
    
    _________________________________
    Parameter:
    dataset -> The give dataset for calculating posterior distribution
    _________________________________
    Return:
    posterior -> A dictionary represents the posterior distribution, that is P(xi|cj)
    """
    # create the framework of the 3-D dictionary for posterior,   
    posterior = create_posterior_framework_with_Lapalce_smoothing(dataset)
    
    
    # total number of attributes
    n = len(dataset[0]) - 1
    
    # count the attribute occurence instance by instance
    for instance in dataset:
        # is the class has not be included in the dictionary, add it in.
        the_class = instance[-1]
            
        # feature means a attribute associated with value
        # count feature according to its class and attribute belongs to
        for attribute in range(n):
            feature = instance[attribute]
            posterior.get(attribute).get(the_class)[feature] += 1
    
    # do the last step of Laplace smoothing, divide attribtue count by (#class + # distinct attribute)
    all_classes = posterior.get(0).keys()
    
    for attr in range(n):
        for the_class in all_classes:
            # since we set initial value of each distinct value count to 1, then if we sum up all distinct value
            # then we actually get number_of_distinct_value + number_of_1, due to the initial value 1.
            num_of_classes_plus_num_of_distinct_values = sum(posterior.get(attr).get(the_class).values())
            for distinct_value in posterior.get(attr).get(the_class).keys():
                posterior.get(attr).get(the_class)[distinct_value] /= num_of_classes_plus_num_of_distinct_values
            
    return posterior

In [ ]:
def create_posterior_framework_with_Lapalce_smoothing(dataset):
    '''
    Helper function of posterior_calculator.
    This function intend to create a data structure shown as below:
     {attribute x: 
                {class c: 
                            {attribute_value: 1}
                }
    }
    
    Note that the initial value of each attribute_value is 1, since I apply Laplace smoothing
    _________________________
    Parameter:
    dataset-> given dataset
    _________________________
    Return:
    posterior_framework -> The generated framework described above
    '''
    # the length of each instance => number of attributes in total
    n = len(dataset[0])
        
    # find distinct values of all attributes, include the class attribute
    # The data structure for storing information about distinct values is
    #
    # {attribute_1: [distinct_value_1, ..., distinct_value_n],
    # ....
    #  attribute_n: [distinct_value_1, ..., distinct_value_n],
    #  }
    
    attribute_distinct_value = {}     
    for i in range(n):
        attribute_distinct_value[i] = []
    
    for instance in dataset:
        for attr in range(n):
            feature = instance[attr]
            if feature not in attribute_distinct_value.get(attr):
                attribute_distinct_value[attr].append(feature)
                
                
    # generate posterior_framework depends on attribute_distinct_value
    num_attr = n - 1 # class attribute should be excluded
    posterior_framework = {}     
    for i in range(num_attr):
        posterior_framework[i] = {}
    
    # class attributes processing
    class_distinct_value = attribute_distinct_value.get(n-1)
    
    for attr in posterior_framework.keys():
        for the_class in class_distinct_value:
            posterior_framework.get(attr)[the_class] = {}
    
    # attribute processing
    for attr in posterior_framework.keys():
        for the_class in class_distinct_value:
            for distinct_value in attribute_distinct_value.get(attr):
                posterior_framework.get(attr).get(the_class)[distinct_value] = 1
    
    # All done!
    return posterior_framework

In [ ]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(test_set, prior, posterior):
    # use arg max to find the class with highes probability of generating one test instance, 
    # and do it for each instance
    for test_instance in test_set:
        # arg max
        
    return

In [ ]:
def argmax(test_instance, prior, posterior):
    # get list of all possible class from prior
    classes = prior.keys()
    # I set this to a list, since there could be more than 1 class shre max probability
    curr_argmax_class = [classes[0]]
    curr_argmax_probability = probability_of_class_given_instance_calculator(test_instance, prior, posterior,
                                                                             classes[0])
    
    for the_class in classes[1:]:
        # handle class with same max probability
    

In [172]:
def probability_of_class_given_instance_calculator(instance, prior, posterior, the_class):
    posterior_probablity = 1
    for attr in range(len(instance) - 1):
        feature = instance[attr]
        # handle missing value by ignore it
        if feature == "?":
            pass
        else:
            posterior_probablity *= posterior.get(attr).get(the_class).get(feature)
    return posterior_probablity*prior.get(the_class)

In [ ]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [ ]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain():
    return

In [159]:
car = preprocess("./2019S1-proj1-data_dos/car.csv")
prior, posterior = train(car)

In [162]:
prior.keys()

dict_keys(['unacc', 'acc', 'vgood', 'good'])

In [161]:
posterior

{0: {'unacc': {'vhigh': 0.29736408566721584,
   'high': 0.2677100494233937,
   'med': 0.2215815485996705,
   'low': 0.21334431630971992},
  'acc': {'vhigh': 0.18814432989690721,
   'high': 0.2809278350515464,
   'med': 0.29896907216494845,
   'low': 0.23195876288659795},
  'vgood': {'vhigh': 0.014492753623188406,
   'high': 0.014492753623188406,
   'med': 0.391304347826087,
   'low': 0.5797101449275363},
  'good': {'vhigh': 0.0136986301369863,
   'high': 0.0136986301369863,
   'med': 0.3287671232876712,
   'low': 0.6438356164383562}},
 1: {'unacc': {'vhigh': 0.29736408566721584,
   'high': 0.25947281713344317,
   'med': 0.2215815485996705,
   'low': 0.2215815485996705},
  'acc': {'vhigh': 0.18814432989690721,
   'high': 0.27319587628865977,
   'med': 0.29896907216494845,
   'low': 0.23969072164948454},
  'vgood': {'vhigh': 0.014492753623188406,
   'high': 0.2028985507246377,
   'med': 0.391304347826087,
   'low': 0.391304347826087},
  'good': {'vhigh': 0.0136986301369863,
   'high': 0.

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.